#RAG Demo
Demonstriert die Implementierung des RAG-Pattern mit der OpenAI API.

##Setup

In [3]:
!pip install openai numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


## API-Zugriff konfigurieren

In [4]:
import openai
import os
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

## Beispieltexte erstellen

In [5]:
knowledge_base = [
    "Python ist eine interpretierte, hochgradig abstrahierte Programmiersprache.",
    "Die OpenAI API ermöglicht den Zugriff auf fortschrittliche KI-Modelle.",
    "Machine Learning ist ein Teilgebiet der künstlichen Intelligenz.",
    # Weitere Texte hinzufügen ...
]

## Embeddings erzeugen

In [6]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    print(text)
    print(response.data[0].embedding)
    return response.data[0].embedding

knowledge_embeddings = [get_embedding(doc) for doc in knowledge_base]


Python ist eine interpretierte, hochgradig abstrahierte Programmiersprache.
[0.014475155621767044, -0.002341095358133316, 0.019940396770834923, -0.01581568643450737, 0.00980263203382492, 0.009183925576508045, -0.0025134955067187548, -0.019695492461323738, 0.00926126353442669, -0.03389996290206909, 0.025238072499632835, 0.011710310354828835, 0.0009095309069380164, -0.012651260010898113, -0.005958273075520992, 0.0006742934929206967, 0.03389996290206909, -0.013740441761910915, 0.019283020868897438, 0.023910431191325188, -0.0048852041363716125, 0.016137929633259773, 0.008210752159357071, -0.0019221794791519642, -0.023497959598898888, 0.014178692363202572, -0.008165637962520123, -0.025882558897137642, 0.0044727330096066, -0.03245631605386734, 0.01934747025370598, -0.022260546684265137, -0.017452681437134743, -0.019308801740407944, -0.03970033675432205, 0.0016531066503375769, -0.01581568643450737, -0.014784509316086769, 0.0057455929927527905, 0.011246280744671822, 0.031244682148098946, -0.00

## Benutzerabfrage erstellen / passende Texte finden

In [7]:
def find_most_relevant_document(user_query):
    query_embedding = get_embedding(user_query)
    similarities = cosine_similarity(
        [query_embedding],
        knowledge_embeddings
    )
    most_relevant_index = np.argmax(similarities)
    return knowledge_base[most_relevant_index]

user_query = "Was ist maschinelles Lernen?"
relevant_document = find_most_relevant_document(user_query)
print(f"Relevantestes Dokument: {relevant_document}")

Was ist maschinelles Lernen?
[-0.020446959882974625, 0.014642491936683655, 0.00896934699267149, -0.011713993735611439, -0.010518955998122692, 0.0058734118938446045, -0.00515441969037056, 0.01623149774968624, -0.019330715760588646, -0.028996072709560394, 0.018831688910722733, 0.024018939584493637, -0.01623149774968624, 0.002644512802362442, 0.008949648588895798, 0.009711320511996746, 0.04740753024816513, 0.00233262125402689, 0.01480007916688919, -0.0023670934606343508, -0.013302999548614025, 0.0062378328293561935, 0.025319034233689308, -0.0371643528342247, -0.02172078937292099, -0.006024433299899101, 0.011313458904623985, -0.02218041941523552, -0.0010013362625613809, -0.012002903036773205, 0.03451162949204445, -0.014130332507193089, 0.008627907373011112, -0.010807866230607033, -0.027157554402947426, 0.000507644668687135, 0.015062724240124226, 0.006579271983355284, 0.012797405943274498, 0.009376447647809982, 0.02954762801527977, 0.013972745276987553, 0.0008872496546246111, -0.01992166787

## Antwort generieren

In [8]:
def generate_response(user_query, context):
    prompt = (
        f"Beantworte die Frage basierend auf dem unten stehenden Kontext. \
        Wenn die Frage nicht mit den bereitgestellten Informationen beantwortet werden kann, antworte mit 'Ich weiß es nicht'.\n\n"
        f"Kontext: {context}\n\n"
        f"Frage: {user_query}"
    )
    messages = [
        {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content

response = generate_response(user_query, relevant_document)
print(f"Antwort: {response}")

Antwort: Maschinelles Lernen ist ein Teilgebiet der künstlichen Intelligenz, das sich damit befasst, wie Computer Algorithmen und Modelle entwickeln können, um aus Daten zu lernen und Vorhersagen oder Entscheidungen zu treffen, ohne explizit programmiert zu werden. Es ermöglicht Maschinen, Muster in Daten zu erkennen und daraus zu lernen, um zukünftige Probleme zu lösen oder Aufgaben zu erledigen.
